In [1]:
import pandas as pd
import numpy as np
import os, sys, glob, inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import epri_mc_lib as mc
from importlib import reload
reload(mc)


<module 'epri_mc_lib' from '/home/nick/Bureau/Programming/S2DS/Oct20_EPRI/Task1/NB/epri_mc_lib.py'>

## Generate artificial data

This notebook generates artifical data from sample measurements. The new samples are drawn from a normal distribution with a mean and standard deviation based on the mean/median and standard deviation of every feature. 

In [2]:
data_path = "../../Data/Merged_data"
merged_data = pd.read_csv(os.path.join(data_path, 'ALL_TUBE_PIPE_merge_1.csv'), 
                 index_col=0)


In [3]:
num_samples_generated = 1000

For now we're throwing out the pipe dating and only working with tubes.

In [4]:
merged_tubes = merged_data.iloc[0:16,].dropna(axis=1)


In [5]:
merged_tubes

,TEP_mean_uV_C,TEP_error_uV_C,Absorption_avg_500,Absorption_std_500,backscatter_avg,backscatter_std,Absorption_avg_50,Absorption_std_50,A,A std,...,Absorption_std_200,median_CF,mean_CF,std_CF,median_perm,mean_perm,std_perm,median_MBN,mean_MBN,std_MBN
T_B1,12.947951,0.073112,0.000234,0.000018,0.0175,0.0042,0.001264,0.000519,4.410000e-13,4.090000e-07,...,0.000094,1.1000,1.100636,0.013418,68.50,68.300000,1.115347,0.4730,0.483909,0.127282
T_B2,12.920068,0.077814,0.000571,0.000139,0.0240,0.0057,0.002486,0.000816,1.520000e-06,1.510000e-06,...,0.000234,1.1095,1.107818,0.015721,66.45,66.336364,0.890250,0.6305,0.602455,0.110443
T_B3,12.932865,0.088762,0.000244,0.000030,0.0134,0.0028,0.001168,0.000555,4.270000e-13,5.640000e-07,...,0.000078,1.0795,1.079909,0.016489,68.40,68.390909,0.770006,0.5765,0.592364,0.178157
T_B4,10.763517,0.071173,0.000379,0.000100,0.0146,0.0031,0.002084,0.000648,7.580000e-07,1.000000e-06,...,0.000136,4.5140,4.509000,0.032860,32.90,32.909091,0.434637,0.5005,0.513182,0.052547
T_B5,12.942958,0.074808,0.000230,0.000033,0.0182,0.0040,0.001141,0.000447,3.970000e-13,4.670000e-07,...,0.000088,1.4290,1.432636,0.017591,64.95,65.200000,1.173882,0.5380,0.534909,0.042655
T_B6,11.037827,0.090636,0.000240,0.000026,0.0148,0.0031,0.001323,0.000574,1.350000e-12,3.500000e-07,...,0.000110,4.6720,4.704636,0.068887,34.05,33.936364,0.881218,0.3010,0.306636,0.030725
T_B7,12.882150,0.099066,0.000406,0.000084,0.0300,0.0087,0.001908,0.000926,4.820000e-13,1.250000e-06,...,0.000080,0.6580,0.678818,0.059461,103.55,100.081818,6.020103,0.6075,0.606455,0.052569
T_B8,12.257788,0.080920,0.000319,0.000076,0.0269,0.0062,0.000933,0.000304,4.420000e-12,4.410000e-07,...,0.000091,2.0400,2.101455,0.372001,57.80,57.127273,3.940835,0.3745,0.413364,0.223698
T_AR,12.947031,0.089787,0.000283,0.000023,0.0179,0.0038,0.001257,0.000210,4.430000e-13,5.060000e-07,...,0.000081,1.2355,1.236364,0.028040,61.85,61.981818,0.435473,0.5940,0.594636,0.082165
T_N,10.805126,0.076842,0.000443,0.000075,0.0253,0.0054,0.003128,0.001147,1.440000e-07,1.730000e-06,...,0.000062,4.4960,4.503364,0.032598,33.40,33.290909,0.291392,0.5915,0.588000,0.070061


In [6]:
final_generated_data = pd.DataFrame(columns = ["Condition"] + mc.data_generation_values)

for index, row in merged_tubes.iterrows():
    generated_samples = pd.DataFrame(num_samples_generated * [index])
    generated_samples.columns = ["Condition"]
    for value_col, std_col in zip(mc.data_generation_values, mc.data_generation_stds):
        generated_samples[value_col] = pd.DataFrame(np.random.normal(merged_tubes.loc[index,value_col], 
              merged_tubes.loc[index,std_col], num_samples_generated))
    final_generated_data = final_generated_data.append(generated_samples, ignore_index = True)

print(final_generated_data)

      Condition  TEP_mean_uV_C  Absorption_avg_500  backscatter_avg  \
0          T_B1      12.873562            0.000233         0.020457   
1          T_B1      12.959031            0.000241         0.018388   
2          T_B1      13.129509            0.000205         0.019468   
3          T_B1      13.023750            0.000245         0.018078   
4          T_B1      12.906452            0.000259         0.021694   
...         ...            ...                 ...              ...   
15995   T_HAZ_T      12.930371            0.000318         0.029765   
15996   T_HAZ_T      13.026200            0.000403         0.015754   
15997   T_HAZ_T      13.018111            0.000283         0.025623   
15998   T_HAZ_T      13.074087            0.000311         0.015163   
15999   T_HAZ_T      13.063618            0.000225         0.019513   

       Absorption_avg_50             A         B         p  \
0               0.000991 -3.054633e-07  0.000989  0.220862   
1               0.00118

In [7]:
final_generated_data.to_csv(os.path.join(data_path, 'ALL_TUBE_PIPE_simulated.csv'), index=False)